In [2]:
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# because I "don't" know the underlying distribution of the data,
# I'm choosing SIZE ssplit points randomly from the data

if rank==0:
    data = np.random(10000)
    
    

In [54]:
size=10
# choose an integer. 
# The higher the integer, the more even the distribution over processes
# should be.
eveness_parameter = 2

data = np.random.random(100)
splitters = np.random.choice(data, replace=False, size=(size-1)*eveness_parameter)
splitters = list(np.reshape(np.sort(splitters), (size-1, eveness_parameter)).mean(axis=1))
low_highs = list(zip([np.NINF] + splitters, splitters + [np.Inf]))

def is_in_range(x, low_high):
    return x <= low_high[0] and x < low_high[1]
def get_bucket_number(x, low_highs=low_highs):
    for i, lh in enumerate(low_highs):
        if is_in_range(x, lh):
            return i
    return i

In [59]:
bucket_numbers = np.array(list(map(get_bucket_number, data)))

for i in range(size):
    # send bucket_numbers
    comm.Isend(data[bucket_numbers==i], i)
    
data = np.empty(1000) # must this be bigger?
comm.Recv(data, source=0)
comm.Gather(data,data,root=0)

if rank==0:
    print(data)

In [69]:
!mkdir assignment11

In [248]:
%%file assignment11/parallel_sorter.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# because I "don't" know the underlying distribution of the data,
# I'm choosing SIZE ssplit points randomly from the data

from numpy.random import permutation

if rank==0:
    data = np.random.random(10000)
    # For a sanity check, I have also used thes
    
    data = np.random.randint(0, 11, 10000)
    data = np.random.randint(0, 11, 10000)
    data = np.array(data, float)
    orig_len_of_data = len(data)
    print(data.dtype)
    #for a sanity check, ensure these hashes add up at end
    sum_data = np.sum(data)
    sum_data2 = np.sum(data**2)
    
    # choose an integer for eveness of buckets
    # The higher the integer, the more even the distribution 
    # over processes should be, but the longer rank 0 spends
    # before distributing
    eveness_parameter = 2

    splitters = np.random.choice(data, replace=True, size=(size-1)*eveness_parameter)
    splitters = list(np.reshape(np.sort(splitters), (size-1, eveness_parameter)).mean(axis=1))
    low_highs = list(zip([np.NINF] + splitters, splitters + [np.Inf]))
    
    def is_in_range(x, low_high):
        return low_high[0] <= x and x < low_high[1]
    def get_bucket_number(x, low_highs=low_highs):
        # this ensures each datapoint will only get sent to one bucket
        # even if there are ties
        for i, lh in enumerate(low_highs):
            if is_in_range(x, lh):
                return i
        return i
    bucket_numbers = np.array(list(map(get_bucket_number, data)))
    
    len_per_bucket = []
    for i in range(size):
        data_to_send = data[bucket_numbers==i]
        len_per_bucket.append(len(data_to_send))
        comm.isend(obj=(len(data_to_send), data.dtype), dest=i, tag=0)
        comm.Isend(data_to_send, i, tag=1)

length, dtype = comm.recv(source=0, tag=0)
print('rank ', rank, ' received ', length)
data = np.full(length, -5, dtype=dtype)
comm.Recv(data, source=0, tag=1)

#print("rank ", rank, " got ", data)
data = np.sort(data)

# I believe that gather always gets it in rank order
#Comm.Gatherv(choice sendbuf[, choice recvbuf, tuple_int sendcounts, tuple_int displacements, MPI_Datatype sendtype], root=0)
if rank==0:
    x_recv = np.full(np.nan, dtype=dtype)
    comm.Gatherv(data, [x_recv, ], root=0)

if rank==0:
    print(data)
    data = np.concatenate(data)
    print(np.array2string(data, separator=','))
    #check if sorted
    print(np.max(data))
    print(data[np.logical_not(np.isclose(np.sort(data), data))])
    print(np.sort(data)[np.logical_not(np.isclose(np.sort(data), data))])

    assert np.all(np.sort(data) == data)
    
    # check data is the size we were expecting
    assert len(data)== orig_len_of_data
    
    # check data is probably exactly the same as what was put in
    # by checking first and second moment
    # we have to use np.isclose(), because, oddly, precision was being lost
    assert np.isclose(np.sum(data), sum_data)
    assert np.isclose(np.sum(data**2), sum_data2)


Overwriting assignment11/parallel_sorter.py


In [253]:
!mpiexec -n 10 python assignment11/parallel_sorter.py

Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignment11/parallel_sorter.py", line 68, in <module>
    data_local)
NameError: name 'data_local' is not defined
Traceback (most recent call last):
  File "assignmen

In [50]:
%%file assignment11/parallel_sorter.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# because I "don't" know the underlying distribution of the data,
# I'm choosing SIZE split points randomly from the data

from numpy.random import permutation

if rank==0:
    data = np.random.random(1000)
    # For a sanity check, I have also used these
    #  data = np.random.random(5)
    #  data = np.random.randint(0, 11, 10000)
    data = np.array(data, float)
        
    #for a sanity check, ensure these hashes match up at end
    orig_len_of_data = len(data)
    sum_data = np.sum(data)
    sum_data2 = np.sum(data**2)
    
    # choose an integer for eveness of buckets
    # The higher the integer, the more even the distribution 
    # over processes should be, but the longer rank 0 spends
    # before distributing
    eveness_parameter = 5

    splitters = np.random.choice(data, replace=True, size=(size-1)*eveness_parameter)
    splitters = list(np.reshape(np.sort(splitters), (size-1, eveness_parameter)).mean(axis=1))
    low_highs = list(zip([np.NINF] + splitters, splitters + [np.Inf]))
    
    def is_in_range(x, low_high):
        return low_high[0] <= x and x < low_high[1]
    def get_bucket_number(x, low_highs=low_highs):
        # this ensures each datapoint will only get sent to one bucket
        # even if there are ties
        for i, lh in enumerate(low_highs):
            if is_in_range(x, lh):
                return i
        return i
    bucket_numbers = np.array(list(map(get_bucket_number, data)))
    
    # since collective communication is faster, I'm going to first do a
    # rough reshuffling.
    # then use Scatterv and Gatherv
    len_per_bucket = []
    roughcut_data = data.copy() * np.nan
    for i in range(size):
        d = data[bucket_numbers==i]
        roughcut_data[ int(np.sum(len_per_bucket)): int(np.sum(len_per_bucket)) + len(d)] = d
        len_per_bucket.append(len(d))
    del data
    displacements = np.array(np.cumsum([0] + len_per_bucket)[0:size], dtype=int)
    len_per_bucket, displacements = tuple(len_per_bucket), tuple(displacements)
    
# broadcast size and displacements
if rank!=0:
    len_per_bucket, displacements = None, None
len_per_bucket, displacements = comm.bcast((len_per_bucket, displacements), root=0)

# prepare buffers of right size
data_local = np.full(len_per_bucket[rank],
                     np.nan,
                    dtype=float)

if rank!=0:
    roughcut_data = None
comm.Scatterv([roughcut_data,
               tuple(len_per_bucket),
               tuple(displacements),
              MPI.DOUBLE
              ],
             data_local)

# do the actual work
data_local = np.sort(data_local)

# gather
if rank==0:
    data = np.zeros(  int(sum(len_per_bucket)) )
else: 
    data = None
comm.Gatherv(data_local, 
             [data,
               tuple(len_per_bucket),
               tuple(displacements),
              MPI.DOUBLE
              ],
            )
    
if rank==0:
    # check data is sorted
    assert np.alltrue(np.sort(data)==data)
    
    # check data is the size we were expecting
    assert len(data)== orig_len_of_data
    
    # check data is probably exactly the same as what was put in
    # by checking first and second moment
    # we have to use np.isclose(), because, oddly, precision was 
    # being lost, probably because of float -> DOUBLE conversion
    assert np.isclose(np.sum(data), sum_data)
    assert np.isclose(np.sum(data**2), sum_data2)
    
    print(data)

Overwriting assignment11/parallel_sorter.py


In [52]:
!mpiexec -n 10 python assignment11/parallel_sorter.py

[ 0.00231326  0.00282741  0.00318615  0.00339956  0.00339986  0.00399505
  0.00441655  0.00575978  0.00602697  0.00663936  0.00737106  0.00785688
  0.00850719  0.00913703  0.00949628  0.01124046  0.01281522  0.01292013
  0.01501309  0.01511138  0.01607246  0.0165139   0.01852581  0.01905302
  0.0202548   0.02073103  0.02074154  0.0207514   0.02130479  0.02255221
  0.02270398  0.02289058  0.0235839   0.02385224  0.02426018  0.02533978
  0.0263802   0.02684541  0.02686903  0.02833508  0.02872822  0.02950672
  0.03035553  0.03171401  0.03292717  0.03387055  0.03541159  0.03669046
  0.03905966  0.03988911  0.04037631  0.04176155  0.04206632  0.04294381
  0.04297658  0.04322214  0.04422136  0.0459712   0.04670334  0.04711243
  0.05062254  0.05108987  0.05125922  0.05128157  0.05173529  0.05206755
  0.05246721  0.05284194  0.05394062  0.05579473  0.05771019  0.05779342
  0.05802577  0.05816103  0.05855138  0.06184328  0.06575805  0.06790196
  0.06812159  0.07113877  0.07315175  0.073998    0

In [39]:
d0 = np.array([ 0.5247752 , 0.18622806, 0.54947639, 0.92271659, 0.5086278 , 0.90435002,
  0.16109585, 0.95567168, 0.5436641 , 0.13356303, 0.01969248, 0.79027124,
  0.34667422, 0.01686329, 0.66121433, 0.24973518, 0.24354808, 0.86229035,
  0.97409281, 0.22661161])
d1 = np.array([ 0.01686329, 0.01969248, 0.13356303, 0.16109585, 0.18622806, 0.22661161,
  0.24354808, 0.24973518, 0.34667422, 0.5086278 , 0.5247752 , 0.5436641 ,
  0.54947639, 0.66121433, 0.79027124, 0.86229035, 0.90435002, 0.92271659,
  0.95567168, 0.97409281])

d0.sum(), d1.sum()

(d0**2).sum(), (d1**2).sum()

NameError: name 'np' is not defined

In [251]:
np.cumsum((1,2,3))

array([1, 3, 6])

In [117]:
from numpy.random import permutation

size = 10
orig_len_of_data = 100
data = np.random.random(orig_len_of_data)
data = np.random.randint(0, 11, 100)

#for a sanity check, ensure these hashes add up at end
sum_data = np.sum(data)
sum_data2 = np.sum(data**2)

# choose an integer for eveness of buckets
# The higher the integer, the more even the distribution 
# over processes should be, but the longer rank 0 spends
# before distributing
eveness_parameter = 2

splitters = np.random.choice(data, replace=False, size=(size-1)*eveness_parameter)
splitters = list(np.reshape(np.sort(splitters), (size-1, eveness_parameter)).mean(axis=1))
low_highs = list(zip([np.NINF] + splitters, splitters + [np.Inf]))

print(low_highs)
def is_in_range(x, low_high):
    return low_high[0] <= x and x < low_high[1]
def get_bucket_number(x, low_highs=low_highs):
    # this ensures each datapoint will only get sent to one bucket
    # even if there are ties
    # the permutation ensures load balancing
    for i, lh in low_highs:
        if is_in_range(x, lh):
            return i
    return i
bucket_numbers = np.array(list(map(get_bucket_number, data)))

[(-inf, 0.0), (0.0, 0.5), (0.5, 2.5), (2.5, 3.0), (3.0, 3.5), (3.5, 4.0), (4.0, 5.5), (5.5, 7.5), (7.5, 9.0), (9.0, inf)]


IndexError: invalid index to scalar variable.

In [118]:
permutation()

TypeError: permutation() takes exactly one argument (0 given)